In [10]:
import sys
import os
import requests
import threading
import time
import json
import re
import pandas
import collections
from bs4 import BeautifulSoup as soup
from datetime import datetime

try:
    from HTMLParser import HTMLParser  # Python 2
except:
    from html.parser import HTMLParser # Python 3

h = HTMLParser()

class SCRAPER:

    URL = "https://www.sec.gov/cgi-bin/browse-edgar?CIK=%s&start=%d&count=100"
    LOCK = threading.Semaphore( value=1 )
    THREADS = 0
    CCOUNT = 0
    ERRORS = 0
    FILECOUNT = 0
    FILES = True
    LINKS = []
    LOOPER = []
    DATA = {}
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:64.0) Gecko/20100101 Firefox/64.0',
    }

    def __init__(self, _ticker, _howmany, _threads=5, _verbose=False):
        ''' A simple and straightforward Scraper! '''
        self.ticker = _ticker
        self.count  = _howmany
        self.max    = _threads   ## More threads more speed except for the file accumulation process  ##
        self.verbose = _verbose  ## Verbose Mode. Print Messages!  ##

    def table(self, _html):
        links, html = [], soup( _html )
        table = html.find( "table", attrs={ 'class': 'tableFile2' } )

        ## Extracting Data from tables ##
        if table:
            rows = table.findChildren( "tr" )[1:]
            for row in rows:
                ( file, format, desc, date, number ) = row.findChildren( "td" )
                if file.text == "10-Q" or file.text == "10-q" or file.text == "10q" or file.text == "10Q":
                    intdata = format.findChild( "a", attrs={ 'id': 'interactiveDataBtn' } )
                    if intdata:
                        links.append( (date.text, "https://www.sec.gov%s" % intdata.get( "href" )) )
                    else:
                        if self.verbose:
                            print("[!] Received 10Q file without interactive data link. ")
            return links
        else:
            self.FILES = False ## Stop if no table is detected ##

    def request(self):
        url = self.URL % ( self.ticker, self.CCOUNT )
        for n in range( 3 ):                             ##  Retry three times in case an error occurs  ##
            try:
                r = requests.get( url, headers=self.HEADERS )
                if r.status_code == 200:
                    self.CCOUNT += 100
                    q10links = self.table( r.text )
                    self.LINKS += q10links
                    if self.verbose:
                        print("[*] Received Number of Files: %d Requested Counter: %d" % (len( q10links ), self.CCOUNT))
                    self.FILECOUNT += len( q10links )
                    if self.FILECOUNT >= self.count:
                        self.FILES = False
                else:
                    raise Exception( "[~] Invalid Response Code Received : %d" % r.status_code )
                break
            except KeyboardInterrupt:
                raise KeyboardInterrupt()
            except:
                if self.verbose:
                    print("[!] Failed connection to Server. Check Your Connection. Trying Again!")

    def search(self):
        if self.verbose:
            print("[^] Requesting 10Q files from the Server!")
        while self.FILES:
            self.request(  )
        if self.verbose:
            print("[*] Acquired Number of 10Q files: %d" % ( self.count ))

    def pprequest( self, _link, _type ):
        ''' The main function to extract all financial statements data. It accepts the link to financial document and return it's dataframe as of created by pandas
            DataFrame() object. It is highly flexbile and determines whether there are any colspans in the table or not. If there are, they will be extracted and
            according deleted when the next colspan is encoundered. whether the argument _link is the link to the financial document and _type is the document type.
            _type argument is only for testing purposes. '''
        try:
            r = requests.get( _link, headers=self.HEADERS )
        except:
            if self.verbose:
                print("[!] Error Requesting Financial Statement. Type: %s" % _type)
                return None
            if r.status_code == 200:
                html = soup( r.text )
                table = html.find( "table", attrs={ 'class': 'report' } )
            if table:
                rows, differ, data = table.findChildren( "tr" ), 0, collections.OrderedDict()
                for row in rows:
                    if row.findChild( "th" ):
                        differ += 1
                headerslist = rows[ :differ ]
                tabdatalist = rows[ differ: ]
                for headers in headerslist:
                    headerslist[ headerslist.index( headers ) ] = headers.findChildren( "th" )
                for tabdata in tabdatalist:
                    tabdatalist[ tabdatalist.index( tabdata ) ] = tabdata.findChildren( "td" )
                for headers in headerslist:
                    for header in headers:
                        if header.get( "colspan" ) and int(header.get( "colspan" )) > 1:
                            colspan = int( header.get( "colspan" ) )
                            for col in range( colspan ):
                                data[ h.unescape( header.text.encode( "utf8" ) ) + " - " + h.unescape( headerslist[ headerslist.index( headers ) + 1 ][ col ].text.encode( "utf8" )) ] = []
                            del headerslist[ headerslist.index( headers ) + 1 ][ :colspan ]
                            del colspan
                        else: 
                            data[ h.unescape( header.text.encode( "utf8" ) ) ] = []
                indexer = data.items()
                for ( _key, _value ) in indexer:
                    for tabdata in tabdatalist:
                        try:
                            data[ _key ].append( h.unescape( tabdata[ indexer.index( ( _key, _value ) ) ].text.encode( "utf8" ) ) )
                        except IndexError:
                            data[ _key ].append( "" )
                return pandas.DataFrame( data, index=range( len( tabdatalist ) ) )
            else:
                return pandas.DataFrame()

    def prequest( self, _date, _link ):
        self.THREADS += 1

        if self.verbose:
            self.LOCK.acquire()
            print("[^] Requesting Q10 File! Date: %s" % _date)
            self.LOCK.release()

        for n in range(2):
            r = requests.get( _link )
            if r.status_code == 200:
                income = "https://www.sec.gov" + re.search( r"reports\[2\+1\] = \"(.*?)\";", r.text ).groups()[0]  # Income Document Link #
                blance = "https://www.sec.gov" + re.search( r"reports\[4\+1\] = \"(.*?)\";", r.text ).groups()[0]  # Balance Document Link #
                otflow = "https://www.sec.gov" + re.search( r"reports\[6\+1\] = \"(.*?)\";", r.text ).groups()[0]  # Cash Flow Document Link #

                income = self.pprequest( income, "i" )
                blance = self.pprequest( blance, "b" )
                otflow = self.pprequest( otflow, "f" )

                if self.verbose:
                    self.LOCK.acquire()
                    print("[*] Received Q10 File. Date: %s" % _date)
                    self.LOCK.release()

                self.DATA[ _date ] = ( income, blance, otflow )
            else:
                if self.verbose:
                    print("[!] Invalid Response Code Received for link: %s" % _link)
            break

        self.THREADS -= 1

    def sort(self):
        ''' This is an object to sort the found files according to their publishing date '''
        dateobjs = []
        for ( date, link ) in self.LINKS:
            dateobjs.append( datetime.strptime( date.encode( "utf8" ), "%Y-%m-%d" ) )
        dateobjs = sorted( dateobjs, reverse=True )[ :self.count ]
        for dateobj in dateobjs:
            for ( date, link ) in self.LINKS:
                if date == dateobj.strftime( "%Y-%m-%d" ):
                    self.LOOPER.append( ( date, link ) )

    def scrape(self):
        for (date, link) in self.LOOPER:
            _t = threading.Thread( target=self.prequest, args=( date, link ) )
            _t.daemon = True
            _t.start()

            while self.THREADS >= self.max:
                time.sleep( 1 )

        while self.THREADS > 0:
            time.sleep( 1 )
        if self.verbose:
            print("[<] Done!")
        return self.DATA

    def acquire(self):
        ''' Function to execute other three functions: search(), sort() and scrape() in sequence '''
        self.search()
        self.sort()
        self.scrape()

        return self.DATA